In [1]:
import os
from functools import partial
from itertools import product

import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score

from NEExT.collections import EgonetCollection
from NEExT.io import GraphIO
from NEExT.ml_models import OutlierDataset

%reload_ext autoreload
%autoreload 2

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from NEExT.experiment_utils.data_loading import initialize_graph, semi_supervised_set
from NEExT.experiment_utils.embed import build_embeddings, build_features
from NEExT.experiment_utils.models import score_unlabeled_gt
from NEExT.ml_models import CosineOutlierDetector


def objective(trial: optuna.Trial, model: str):
    structural_embedding_dimension = trial.suggest_int("structural_embedding_dimension", 1, 20)
    strategy = "structural_embedding"
    feature_embedding_dimension = 1

    embeddings = build_embeddings(
        subgraph_collection,
        structural_features,
        features,
        strategy=strategy,
        structural_embedding_dimension=structural_embedding_dimension,
        feature_embedding_dimension=feature_embedding_dimension,
    )
    dataset = OutlierDataset(subgraph_collection, embeddings, standardize=False)
    if model == "cosine":
        top_k = trial.suggest_int("top_k", 1, 20)
        detector = CosineOutlierDetector(top_k=top_k)
    elif model == "knn":
        n_neighbors = trial.suggest_int("n_neighbors", 1, 50)
        detector = KNeighborsClassifier(n_neighbors=n_neighbors)

    metric = cross_val_score(
        detector,
        dataset.X_labeled,
        dataset.y_labeled,
        cv=StratifiedKFold(5),
        n_jobs=-1,
        scoring=make_scorer(roc_auc_score),
    )
    return metric.mean()

In [3]:
graph_io = GraphIO()
path = "../local/data/simulated"
name = "abcdo_data_1000_200_0.1"
frac = (0.8, 0.8)

graph_data = {
    "name": name,
    "edge_file_path": f"{path}/{name}/edges.csv",
    "node_graph_mapping_file_path": f"{path}/{name}/graph_mapping.csv",
    "features_file_path": f"{path}/{name}/features.csv",
    "skip_features": ["community_id", "random_community_feature"],
    "feature_list": [],
    "target": "is_outlier",
}
edges_df, mapping_df, features_df, ground_truth_df = initialize_graph(graph_data, frac)

In [4]:
graph_collection = graph_io.load_from_dfs(
    edges_df=edges_df,
    node_graph_df=mapping_df,
    node_features_df=features_df,
    graph_type="igraph",
)
subgraph_collection = EgonetCollection()
subgraph_collection.create_egonets_from_graphs(
    graph_collection=graph_collection,
    egonet_target=graph_data["target"],
    egonet_algorithm="k_hop_egonet",
    skip_features=graph_data["skip_features"],
    max_hop_length=1,
)
structural_features, features = build_features(subgraph_collection, feature_vector_length=6, feature_list=graph_data["feature_list"])

In [5]:
model = 'cosine'
study = optuna.create_study(direction="maximize")
study.optimize(partial(objective, model=model), n_trials=50, n_jobs=5)

[I 2025-03-23 18:12:33,413] A new study created in memory with name: no-name-f3a130cc-4592-4d3e-a345-5724a16885c1


[I 2025-03-23 18:12:51,379] Trial 1 finished with value: 0.6375 and parameters: {'structural_embedding_dimension': 6, 'top_k': 16}. Best is trial 1 with value: 0.6375.
[I 2025-03-23 18:12:51,385] Trial 3 finished with value: 0.58125 and parameters: {'structural_embedding_dimension': 12, 'top_k': 20}. Best is trial 1 with value: 0.6375.
[I 2025-03-23 18:12:51,799] Trial 2 finished with value: 0.6875 and parameters: {'structural_embedding_dimension': 9, 'top_k': 19}. Best is trial 2 with value: 0.6875.
[I 2025-03-23 18:12:52,147] Trial 0 finished with value: 0.609375 and parameters: {'structural_embedding_dimension': 4, 'top_k': 20}. Best is trial 2 with value: 0.6875.
[I 2025-03-23 18:12:52,670] Trial 5 finished with value: 0.759375 and parameters: {'structural_embedding_dimension': 12, 'top_k': 13}. Best is trial 5 with value: 0.759375.
[I 2025-03-23 18:12:52,963] Trial 4 finished with value: 0.578125 and parameters: {'structural_embedding_dimension': 2, 'top_k': 11}. Best is trial 5 w

In [ ]:
structural_embedding_dimension = study.best_params['structural_embedding_dimension']
strategy = "structural_embedding"
feature_embedding_dimension = 1
    
embeddings = build_embeddings(
    subgraph_collection,
    structural_features,
    features,
    strategy=strategy,
    structural_embedding_dimension=structural_embedding_dimension,
    feature_embedding_dimension=feature_embedding_dimension,
)
dataset = OutlierDataset(subgraph_collection, embeddings, standardize=False)
if model == "cosine":
    new_var = study.best_params['top_k']
    detector = CosineOutlierDetector(top_k=new_var)
elif model == "knn":
    n_neighbors = study.best_params['n_neighbors']
    detector = KNeighborsClassifier(n_neighbors=n_neighbors)
    
score_unlabeled_gt(dataset, detector, ground_truth_df)

0.76875